#### Understanding Swing Length

In [1]:
import pandas as pd
import pingouin as pg
import os

import os
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/Hitting Analysis/data')

In [2]:
pitch = pd.read_csv('./full/pitch.csv')

mean swing length and mean swing tilt

In [3]:
length_tilt = pd.DataFrame()
pitch_zone = pitch[pitch['statcast_zone'] == 5]
length_tilt = pitch_zone.groupby(['batter', 'season'])[['tilt', 'swing_length']].mean().reset_index()
length_tilt = length_tilt.dropna()

In [4]:
print(f"corr between tilt and length {length_tilt['swing_length'].corr(length_tilt['tilt'])}")

corr between tilt and length -0.1726427187037003


wobacon and swing length 

In [8]:
length_woba = pd.DataFrame()
pitch_hit = pitch[pitch['description'] == 'hit_into_play']
count_df = pitch_hit.groupby(['batter', 'season']).size().reset_index(name='pitch_count')
length_woba = pitch_hit.groupby(['batter', 'season'])[['xwoba', 'swing_length', 'attack_angle', 'bat_speed']].mean().reset_index()
length_woba = pd.merge(length_woba, count_df, on=['batter', 'season'])
length_woba = length_woba.dropna()
# length_woba = length_woba[length_woba['pitch_count'] > 300]

In [9]:
pcorr = pg.partial_corr(data=length_woba, x='swing_length', y='attack_angle', covar='bat_speed')
corr = print(f"full corr {length_woba['swing_length'].corr(length_woba['xwoba'])}")
print(pcorr)

full corr 0.2767411083481431
            n         r         CI95%         p-val
pearson  1200  0.297632  [0.25, 0.35]  5.965523e-26


swing length and attack direction

In [ ]:
length_attack = pd.DataFrame()
length_attack = pitch_zone.groupby(['batter', 'season'])[['attack_angle', 'swing_length', 'attack_direction',]].mean().reset_index()
length_attack = pd.merge(length_attack, count_df, on=['batter', 'season'])
length_attack = length_attack.dropna()

In [11]:
print(f"length and attack {length_attack['swing_length'].corr(length_attack['attack_angle'])}")
print(f"length and direction {length_attack['swing_length'].corr(length_attack['attack_direction'])}")


length and attack 0.4959363207444794
length and direction -0.2811173512618194


tilt vs swing and miss

In [26]:
pitch_break = pitch[pitch['pitch_name'].isin(['Curveball', 'Knuckle Curve', 'Slow Curve', 'Slider', 'Sweeper', 'Slurve', 'Screwball'])]
tilt_miss = pitch_break.groupby(['batter', 'season'])[['tilt', 'swing_length']].mean().reset_index()
total_pitches = pitch_break.groupby(['batter', 'season']).size().reset_index(name='total_pitches')
swing_miss_pitches = pitch_break[pitch_break['description'].isin(['swinging_strike', 'swinging_strike_blocked'])] \
    .groupby(['batter', 'season']).size().reset_index(name='swing_miss_count')
miss_pct = pd.merge(total_pitches, swing_miss_pitches, on=['batter', 'season'], how='left')
miss_pct['swing_miss_count'] = miss_pct['swing_miss_count'].fillna(0)
miss_pct['miss_pct'] = miss_pct['swing_miss_count'] / miss_pct['total_pitches']
tilt_miss = pd.merge(tilt_miss, miss_pct[['batter', 'season', 'miss_pct', 'total_pitches']], on=['batter', 'season'], how='left')
tilt_miss = tilt_miss.dropna()
tilt_miss = tilt_miss[tilt_miss['total_pitches'] > 250]

In [27]:
# find the mean swing and miss for those in the top 20% of swing tilt and then the mean for those in the bottom 20%
top_20 = tilt_miss[tilt_miss['tilt'] >= tilt_miss['tilt'].quantile(0.8)]
bottom_20 = tilt_miss[tilt_miss['tilt'] <= tilt_miss['tilt'].quantile(0.2)]

mean_miss_top_20 = top_20['miss_pct'].mean()
mean_miss_bottom_20 = bottom_20['miss_pct'].mean()

print(f"Mean miss_pct for top 20% tilt: {mean_miss_top_20:.3f}")
print(f"Mean miss_pct for bottom 20% tilt: {mean_miss_bottom_20:.3f}")

Mean miss_pct for top 20% tilt: 0.158
Mean miss_pct for bottom 20% tilt: 0.139


In [28]:
pitch_break = pitch[pitch['pitch_name'].isin(['4-Seam Fastball', 'Cutter', 'Sinker', '2-Seam Fastball'])]
tilt_miss = pitch_break.groupby(['batter', 'season'])[['tilt', 'swing_length']].mean().reset_index()
total_pitches = pitch_break.groupby(['batter', 'season']).size().reset_index(name='total_pitches')
swing_miss_pitches = pitch_break[pitch_break['description'].isin(['swinging_strike', 'swinging_strike_blocked'])] \
    .groupby(['batter', 'season']).size().reset_index(name='swing_miss_count')
miss_pct = pd.merge(total_pitches, swing_miss_pitches, on=['batter', 'season'], how='left')
miss_pct['swing_miss_count'] = miss_pct['swing_miss_count'].fillna(0)
miss_pct['miss_pct'] = miss_pct['swing_miss_count'] / miss_pct['total_pitches']
tilt_miss = pd.merge(tilt_miss, miss_pct[['batter', 'season', 'miss_pct', 'total_pitches']], on=['batter', 'season'], how='left')
tilt_miss = tilt_miss.dropna()
tilt_miss = tilt_miss[tilt_miss['total_pitches'] > 250]

In [29]:
# find the mean swing and miss for those in the top 20% of swing tilt and then the mean for those in the bottom 20%
top_20 = tilt_miss[tilt_miss['tilt'] >= tilt_miss['tilt'].quantile(0.8)]
bottom_20 = tilt_miss[tilt_miss['tilt'] <= tilt_miss['tilt'].quantile(0.2)]

mean_miss_top_20 = top_20['miss_pct'].mean()
mean_miss_bottom_20 = bottom_20['miss_pct'].mean()

print(f"Mean miss_pct for top 20% tilt: {mean_miss_top_20:.3f}")
print(f"Mean miss_pct for bottom 20% tilt: {mean_miss_bottom_20:.3f}")

Mean miss_pct for top 20% tilt: 0.092
Mean miss_pct for bottom 20% tilt: 0.079
